# Create crowd input workflow


1.) Create all questions of a run

- this should not be changed (only created once)
- if the question formulations change, we create a new run
- identifiers should not change
- create unique ids here
- we draw from this selection for creating batches 


2.) Create batch

- draw from all questions of a run
- only select properties in the experiment group
- always make sure that a batch contains the full set of questions of a pair 
- add prolific url 
- check prolific_input for already posed pairs
- write to file without header


Run: 

`python create_questions.py [run]`

`python create_batch.py [url] [batch/test]`

## Fix csv splitting issue

In [69]:
import csv

run = 4
path = f'../templates/template-run{run}.csv'
path_backup = f'../templates/template-run{run}-backup.csv'
with open(path) as infile:
    lines = infile.read().strip().split('\n')
    
with open(path_backup, 'w') as outfile:
    outfile.write('\n'.join(lines))

header = lines[0].strip().split('\t')
correct_length = len(header)
print(correct_length)
print(header)

new_lines = []
for line in lines:
    line_l = line.strip().split('\t')
    n_cells = len(line_l)
    if n_cells == correct_length:
        new_lines.append(line.strip())
    elif n_cells == 10:
        repaired_sent = ','.join(line_l[1:3]).strip()
        if repaired_sent[-1] != '.':
            repaired_sent = repaired_sent+'.'
        new_line_l = []
        new_line_l.append(line_l[0])
        new_line_l.append(repaired_sent)
        new_line_l.extend(line_l[3:])
        #print(len(new_line_l))
        #print(new_line_l)
        new_lines.append('\t'.join(new_line_l).strip())   
    elif n_cells == 11:
        repaired_sent = ','.join(line_l[1:4])
        new_line_l = []
        new_line_l.append(line_l[0])
        new_line_l.append(repaired_sent)
        new_line_l.extend(line_l[4:])
        #print(new_line_l)
        #print(len(new_line_l))
        #print(new_line_l)
        new_lines.append('\t'.join(new_line_l).strip())
    else:
        print('missed something')
        #print(line_l)
                         
print('\n checking if lines have the correct lenght\n')
for l in new_lines:
    print(len(l.split('\t')))
    #print(l)
with open(path, 'w') as outfile:
    outfile.write('\n'.join(new_lines))

9
['level', 'question', 'relation', 'perceptual', 'perceptual_scale', 'complex', 'complex_scale', 'parts', 'activities']

 checking if lines have the correct lenght

9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9


# Find example


In [2]:
from utils import read_csv
import glob

# Get example properties
def find_example(target, target_prop_concept):
    ex_files = glob.glob('../examples/*-pairs.csv')
    for f in ex_files:
        with open(f) as infile:
            dl = read_csv(f)
            for d in dl:
                candidates = []
                for k, v in d.items():
                    if k.startswith(target_prop_concept):
                        prop_concept = v
                        candidates.append(prop_concept)
                if target in candidates:
                    print(f'Found {target} in {f}')
                    

target = 'maade_of_plastic'
target_prop_concept = 'prop'
find_example(target, target_prop_concept)

In [15]:
# get all examples for review
from collections import defaultdict

def collect_all_examples(run, relation=None):
    
    f = f'../questions/run{run}-all-restricted_True.csv'
    dl = read_csv(f)
    all_examples = defaultdict(set)
    ex_keys = ['example_pos', 'example_neg']
    
    for d in dl:
        examples = []
        rel = d['relation']
        for k in ex_keys:
            examples.append(d[k])
        all_examples[rel].add(tuple(examples))
    for rel, examples in all_examples.items():
        if rel == None:
            print(rel)
            for ex_pos, ex_neg in examples:
                print(f'pos: {ex_pos}')
                print(f'neg: {ex_neg}')
                print()
        else:
            if rel == relation:
                for ex_pos, ex_neg in examples:
                    print(f'pos: {ex_pos}')
                    print(f'neg: {ex_neg}')
                    print()


run = 4
collect_all_examples(run, relation = 'variability_limited')           

pos: You can find (a/an) pepper which is red. Red is one of a few possible colors (a/an) pepper usually has. There is only a limited range of possible colors.
neg: You can find (a/an) t-shirt which is red. Red is one of a few possible colors (a/an) t-shirt usually has. There is only a limited range of possible colors.

pos: You can find (a/an) car which has (a/an) removable roof. (A/an) car usually either has an removable roof or no removable roof. This distinguishes a certain type of car from others.
neg: You can find (a/an) bike which has (a/an) handle bar. (A/an) bike usually either has an handle bar or no handle bar. This distinguishes a certain type of bike from others.

pos: You can find (a/an) car which has (a/an) emergency light. (A/an) car usually either has an emergency light or no emergency light. This distinguishes a certain type of car from others.
neg: You can find (a/an) car which has (a/an) spare_wheel. (A/an) car usually either has an spare_wheel or no spare_wheel. Thi

# Updating examples or issues in the questions

In [ ]:
# moved to update_questions.py